# 3. Keras Tuner + Analytical Model Ensembling

In [28]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from functools import reduce
from pandas import read_csv
from matplotlib import pyplot

# Import LSTM libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from keras.preprocessing.timeseries import TimeseriesGenerator
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import tensorflow as tf 
tf.keras.preprocessing.timeseries_dataset_from_array

<function keras.src.utils.timeseries_dataset_utils.timeseries_dataset_from_array(data, targets, sequence_length, sequence_stride=1, sampling_rate=1, batch_size=128, shuffle=False, seed=None, start_index=None, end_index=None)>

In [29]:
# Import all datasets
oil_data_clean = pd.read_csv('cleaned_oil.csv')
ppi_data_clean = pd.read_csv('cleaned_ppi.csv')
unemp_data_clean = pd.read_csv('cleaned_unemp.csv')
lcpi_data_clean = pd.read_csv('cleaned_lagged_cpi.csv')
cpi_data_clean = pd.read_csv('cleaned_cpi.csv')

In [30]:
oil_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
ppi_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
unemp_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
lcpi_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)
cpi_data_clean.drop(columns = ['ChangePercentYear', 'Value'], axis = 1, inplace = True)

In [31]:
cpi_data_clean.fillna(method='bfill')
lcpi_data_clean.fillna(method='bfill')
oil_data_clean.fillna(method='bfill')
ppi_data_clean.fillna(method='bfill')
unemp_data_clean.fillna(method='bfill')

C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_5852\728974245.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cpi_data_clean.fillna(method='bfill')
C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_5852\728974245.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lcpi_data_clean.fillna(method='bfill')
C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_5852\728974245.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil_data_clean.fillna(method='bfill')
C:\Users\Suryanshu Choudhary\AppData\Local\Temp\ipykernel_5852\728974245.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ppi_data_clean.fi

,Date,ChangePercentMonth
0,1948-01,0.175000
1,1948-02,0.175000
2,1948-03,-0.042553
3,1948-04,-0.111111
4,1948-05,-0.150000
...,...,...
882,2021-07,-0.065574
883,2021-08,-0.070175
884,2021-09,-0.132075
885,2021-10,-0.065217


In [32]:
oil_data_clean.columns = ['Date','Oil']
oil_data_clean.dropna(inplace = True)
ppi_data_clean.columns = ['Date','PPI']
ppi_data_clean.dropna(inplace = True)
unemp_data_clean.columns = ['Date','Unemployment']
unemp_data_clean.dropna(inplace = True)
lcpi_data_clean.columns = ['Date','LaggedCPI']
lcpi_data_clean.dropna(inplace = True)
cpi_data_clean.columns = ['Date','CPI']
cpi_data_clean.dropna(inplace = True)

dataframe_list = [cpi_data_clean, oil_data_clean, ppi_data_clean, unemp_data_clean, lcpi_data_clean]
# Join all datasets together
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='inner'), dataframe_list)
df_merged.set_index('Date', inplace = True)

In [33]:
values = df_merged.values

In [34]:
# Data Preparation for Ensemble Model
df_merged

,CPI,Oil,PPI,Unemployment,LaggedCPI
Date,,,,,
1983-04,0.007150,0.046464,0.000000,-0.074074,0.000000
1983-05,0.006085,-0.012406,0.003984,-0.020000,0.007150
1983-06,0.003024,0.037355,0.001984,0.040816,0.006085
1983-07,0.004020,0.019758,0.002970,-0.078431,0.003024
1983-08,0.003003,-0.012813,0.004936,-0.021277,0.004020
...,...,...,...,...,...
2021-07,0.004811,0.006533,0.012888,-0.065574,0.009291
2021-08,0.002066,-0.073698,0.006750,-0.070175,0.004811
2021-09,0.002716,0.095328,0.009695,-0.132075,0.002066


In [35]:
df_merged.isnull().sum()

CPI             0
Oil             0
PPI             0
Unemployment    0
LaggedCPI       0
dtype: int64

In [36]:
# Splitting the dataset into train and test sets
df_train = df_merged.iloc[:357]
df_test = df_merged.iloc[357:]

In [37]:
# Use MinMaxScaler to normalise are values by scaling down values between a range of -1 to 1
# Instantiate scaler objects
x_scaler = MinMaxScaler(feature_range=(-1, 1))
y_scaler = MinMaxScaler(feature_range=(-1, 1))

x_train = x_scaler.fit_transform(df_train[['Oil','PPI','Unemployment','LaggedCPI']])
y_train = y_scaler.fit_transform(df_train[['CPI']])

x_test = x_scaler.transform(df_test[['Oil','PPI','Unemployment','LaggedCPI']])
y_test = y_scaler.transform(df_test[['CPI']])

In [38]:
# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(357, 1, 4) (357, 1) (107, 1, 4) (107, 1)


In [39]:
# serializable format
import tensorflow as tf
from tensorflow.keras.models import load_model

# Register mse with TensorFlow's saving utilities
tf.keras.utils.get_custom_objects().update({'mse': tf.keras.losses.mean_squared_error})

# Load the model
kerasModel = load_model(
    'kerastunermodel.h5', 
    custom_objects={'MeanSquaredError': tf.keras.metrics.MeanSquaredError}
)


In [40]:
handcraftedModel = load_model('multivariate.h5')

In [41]:
# Checking if kerasModel is already trained or not
if kerasModel.get_weights():
    print("Best model is already trained.")
else:
    print("Best model needs to be trained.")

Best model is already trained.


In [42]:
# Checking if handcraftedModel is already trained or not
if handcraftedModel.get_weights():
    print("Best model is already trained.")
else:
    print("Best model needs to be trained.")

Best model is already trained.


In [43]:
# Get predictions from the kerasModel
tuner_pred = kerasModel.predict(x_test)

# Get predictions from handcraftedModel
own_pred = handcraftedModel.predict(x_test)

1/4 ━━━━━━━━━━━━━━━━━━━━ 1s 402ms/stepWARNING:tensorflow:6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001F895DFEC00> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step


## Weighted Average Ensembling Method

In [58]:
tuner_weight = 0.5
own_weight = 0.5

ensemble_pred = (tuner_pred * tuner_weight) + (own_pred * own_weight)

In [61]:
# Calculate RMSE
mse = mean_squared_error(df_test['CPI'], ensemble_pred)
rmse = sqrt(mse)
print('Ensemble RMSE:', rmse)

# Calculate MAE
mae = mean_absolute_error(df_test['CPI'], ensemble_pred)
print('Ensemble MAE:', mae)

Ensemble RMSE: 0.40578708079398434
Ensemble MAE: 0.3616907778858559
